In [1]:
# Sample API
import mae6225
#import simulation
mae6225.__version__

'0.1.dev'

In [2]:
# Define grid parameters
nx, ny = 40, 40
xmin, xmax = 0.0, 1.0
ymin, ymax = 0.0, 1.0

Re   = 100
dt   = 0.01
tmax = 10.

# Define cell-centered variable names
center_vars = ['pres', 'divp', 'divc','asol','eror']
face_vars   = ['velc', 'hvar','asol','eror']

# Define boundary conditions for variable pressure and velocity [left, right, bottom, top]
bc_type_pres = {'pres': ['neumann', 'neumann', 'neumann', 'neumann']}
bc_val_pres  = {'pres': [0.0, 0.0, 0.0, 0.0]}

bc_type_u = {'velc': ['dirichlet', 'dirichlet', 'dirichlet', 'dirichlet']}
bc_val_u  = {'velc': [0.0, 0.0, 0.0, 1.0]}

bc_type_v = {'velc': ['dirichlet', 'dirichlet', 'dirichlet', 'dirichlet']}
bc_val_v  = {'velc': [0.0, 0.0, 0.0, 0.0]}


# Create the grid and data
gridc = mae6225.Grid('cell-centered', center_vars,
                    nx, ny, xmin, xmax, ymin, ymax,
                    user_bc_type=bc_type_pres, user_bc_val=bc_val_pres)

gridx = mae6225.Grid('x-face', face_vars,
                    nx, ny, xmin, xmax, ymin, ymax,
                    user_bc_type=bc_type_u, user_bc_val=bc_val_u)

gridy = mae6225.Grid('y-face', face_vars,
                    nx, ny, xmin, xmax, ymin, ymax,
                    user_bc_type=bc_type_v, user_bc_val=bc_val_v)

In [3]:
t  = 0.0
nt = 0

ins_stats = dict()

while(t<=tmax):
    
    # Calculate predicted velocity: u* = dt*H(u^n)
    mae6225.ins.predictor(gridx, gridy, 'velc', 'hvar', Re, dt)
    
    # Calculate RHS for the pressure Poission solver div(u)/dt
    mae6225.ins.divergence(gridc, gridx, gridy, 'velc', 'divp', ifac=dt)
    
    # Solve the pressure Poisson equation 
    ins_stats['ites'],ins_stats['res'] = mae6225.poisson.solve_jacobi(gridc, 'pres', 'divp',
                                          maxiter=10000, tol=1e-9)
    
    # Calculate corrected velocity u^n+1 = u* - dt * grad(P) 
    mae6225.ins.corrector(gridc, gridx, gridy, 'velc', 'pres', dt)
    
    # Calculate divergence of the corrected velocity to display stats
    mae6225.ins.divergence(gridc, gridx, gridy, 'velc', 'divc')
    
    # Calculate stats
    ins_stats.update(mae6225.ins.stats(gridc, gridx, gridy, 'velc', 'pres', 'divc'))
    
    # Display stats
    if(nt%10==0): mae6225.io.display_stats(t,ins_stats)   
    
    t  = t+dt
    nt = nt+1
    

------------ Time = 0.0 ---------------
Number of poisson iterations    : 1
Final poisson residual residual : 0.0
Max, Min, U   : 2.0, 0.0
Max, Min, V   : 0.0, 0.0
Max, Min, P   : 0.0, 0.0
Max, Min, DIV : 0.0, 0.0


------------ Time = 0.09999999999999999 ---------------
Number of poisson iterations    : 5538
Final poisson residual residual : 9.98778261744523e-10
Max, Min, U   : 2.0, -0.1276021295378761
Max, Min, V   : 0.29587932179126736, -0.3038712901239007
Max, Min, P   : 1.2094198143214747, -0.8703529014661785
Max, Min, DIV : 8.958161501482209e-08, -8.95165846843092e-08


------------ Time = 0.20000000000000004 ---------------
Number of poisson iterations    : 4765
Final poisson residual residual : 9.98887455683559e-10
Max, Min, U   : 2.0, -0.14215177739944862
Max, Min, V   : 0.3254680445370629, -0.3762157290232419
Max, Min, P   : 1.2229836202495217, -0.7778045378343613
Max, Min, DIV : 9.060888039869042e-08, -9.040188229947344e-08


------------ Time = 0.3000000000000001 ----------

KeyboardInterrupt: 

In [ ]:
# Plot the analytical solution
mae6225.io.plot_contour(gridc, 'pres')
mae6225.io.plot_vector(gridx,gridy,'velc')